<a href="https://colab.research.google.com/github/lizehrhardt/recaptcha/blob/main/import.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount GoogleDrive where datasets are located

In [27]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
!ls "/content/drive/MyDrive/captcha/large/train"
!pwd

 Bicycle   Bus	 Chimney     Hydrant	  Mountain   Palm    'Traffic Light'
 Bridge    Car	 Crosswalk   Motorcycle   Other      Stairs
/content


Import python packages and declare global variables

In [29]:
import os
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

import matplotlib.pyplot as plt
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [30]:
large_labels = ['Bicycle', 'Bridge', 'Bus', 'Car', 'Chimney', 'Crosswalk', 'Hydrant', 'Motorcycle', 'Mountain', 'Other', 'Palm', 'Stairs', 'Traffic Light']

image_size = 128
batch_size = 32

np_data = 'numpy_data/'

if (not os.path.exists(np_data)):
      os.mkdir(np_data)

large_train_path = "/content/drive/MyDrive/captcha/large/train"
large_test_path = "/content/drive/MyDrive/captcha/large/val"

In [31]:
def save_data(data_dir, file_prefix, labels):
    data = [] 
    
    for label in labels:
      
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                #img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                img_path = os.path.join(path, img)
                img = keras.preprocessing.image.load_img(img_path, target_size=(image_size, image_size))
                img_arr = keras.preprocessing.image.img_to_array(img)
                img_arr = img_arr/255.0
                #img_arr = cv2.imread(os.path.join(path,img),1)
                #resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                #data.append([resized_arr, class_num])
                data.append([img_arr, class_num])
            except Exception as e:
                print(e)
        np.save(np_data+file_prefix+label, data)
    return np.array(data)

In [32]:
def load_data(file_path):
  if (os.path.exists(file_path)):
    return np.load(file_path)
  else:
    return False

In [34]:
large_train_data = save_data(large_train_path,'Large_Train_',large_labels)
large_test_data = save_data(large_test_path,'Large_Test_',large_labels)

/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [11]:
train_datagen = keras.preprocessing.image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(directory=large_train_path, target_size=(image_size, image_size))

Found 16440 images belonging to 13 classes.


In [12]:
validation_datagen = keras.preprocessing.image.ImageDataGenerator()

validation_generator = validation_datagen.flow_from_directory(directory=large_test_path, target_size=(image_size, image_size))

Found 4117 images belonging to 13 classes.


In [36]:
base_model = tf.keras.applications.MobileNet(input_shape=(image_size,image_size,3), include_top=False)

base_model.trainable = False

base_model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(13, activation='sigmoid')
])

base_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

base_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_128 (Functio  (None, 4, 4, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d_7   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_8 (Dense)             (None, 13)                13325     
                                                                 
Total params: 3,242,189
Trainable params: 13,325
Non-trainable params: 3,228,864
_________________________________________________________________


In [37]:
epochs = 15
steps_per_epoch = np.ceil(train_generator.n / batch_size)
validation_steps = np.ceil(validation_generator.n / batch_size)

history = base_model.fit_generator(generator=train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              validation_data=validation_generator,
                              validation_steps=validation_steps)

base_model.save('MobileNet_TransferLearning_Captcha.h5')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == '__main__':


Epoch 1/15
514/514 [==============================] - 270s 519ms/step - loss: 0.2832 - accuracy: 0.2517 - val_loss: 0.2174 - val_accuracy: 0.3427
Epoch 2/15
514/514 [==============================] - 259s 503ms/step - loss: 0.2093 - accuracy: 0.3729 - val_loss: 0.2032 - val_accuracy: 0.3983
Epoch 3/15
514/514 [==============================] - 259s 503ms/step - loss: 0.1992 - accuracy: 0.4083 - val_loss: 0.1967 - val_accuracy: 0.4195
Epoch 4/15
514/514 [==============================] - 267s 519ms/step - loss: 0.1937 - accuracy: 0.4253 - val_loss: 0.1923 - val_accuracy: 0.4348
Epoch 5/15
514/514 [==============================] - 258s 502ms/step - loss: 0.1899 - accuracy: 0.4447 - val_loss: 0.1895 - val_accuracy: 0.4443
Epoch 6/15
514/514 [==============================] - 258s 503ms/step - loss: 0.1870 - accuracy: 0.4549 - val_loss: 0.1870 - val_accuracy: 0.4588
Epoch 7/15
514/514 [==============================] - 258s 502ms/step - loss: 0.1847 - accuracy: 0.4650 - val_loss: 0.1852 -

In [45]:
net_model = tf.keras.applications.MobileNet(input_shape=(image_size,image_size,3), include_top=False)

net_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

net_model.summary()

Model: "mobilenet_1.00_128"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 64, 64, 32)        864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 64, 64, 32)       128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 64, 64, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)       288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 64, 64, 32)       128       
 ation)                                         